In [1]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import load_model

In [3]:
# Load the saved model and label encoder
model = tf.keras.models.load_model('/content/drive/MyDrive/New-Job/cnn_model.h5')
label_encoder = joblib.load('/content/drive/MyDrive/Wail-Projet-F/CNN/Data/label_encoder.pkl')

In [4]:
# Get the model's expected input shape
input_shape = model.input_shape
print(f"Model's expected input shape: {input_shape}")
expected_side_length = input_shape[1]
expected_features = expected_side_length ** 2

Model's expected input shape: (None, 18, 18, 1)


In [ ]:
# Load the test dataset for testing
data = pd.read_csv('/content/drive/MyDrive/New-Job/test_dataset.csv')
print(f"Columns in test data: {data.columns.tolist()}")

Columns in test data: ['FID', 'IID', 'Dimension', 'basic.blocks_max', 'basic.blocks_min', 'basic.cells_filled', 'basic.cells_total', 'basic.costs_fun_evals', 'basic.costs_runtime', 'basic.dim', 'basic.lower_max', 'basic.lower_min', 'basic.minimize_fun', 'basic.objective_max', 'basic.objective_min', 'basic.observations', 'basic.upper_max', 'basic.upper_min', 'bt.mean.attractor_dists.max', 'bt.mean.attractor_dists.mean', 'bt.mean.attractor_dists.median', 'bt.mean.attractor_dists.min', 'bt.mean.basin_intersection.max', 'bt.mean.basin_intersection.mean', 'bt.mean.basin_intersection.median', 'bt.mean.basin_intersection.min', 'bt.mean.basin_range', 'bt.mean.basin_ratio.certain', 'bt.mean.basin_ratio.most_likely', 'bt.mean.basin_ratio.uncertain', 'bt.mean.costs_fun_evals', 'bt.mean.costs_runtime', 'bt.mean.depth', 'bt.mean.leaves', 'bt.mean.levels', 'bt.mean.levels_nodes_ratio', 'bt.min.attractor_dists.max', 'bt.min.attractor_dists.mean', 'bt.min.attractor_dists.median', 'bt.min.attractor_dis

In [ ]:
# Load the original dataset for output
original_data = pd.read_csv('/content/drive/MyDrive/Wail-Projet-F/CNN/Data/dataset.csv')
output_data = original_data[['Dimension', 'FID', 'IID']].copy()

In [ ]:
# Separate features and true labels from the test dataset
columns_to_drop = ['FID', 'IID', 'Dimension', 'ERT', 'min_ERT', 'RELERT']
data_features = data.drop(columns=[col for col in columns_to_drop if col in data.columns], axis=1)
true_labels_numeric = data['Best Algorithm'].values

In [8]:
# Debug: Print unique true labels
print(f"Unique true labels (numeric): {np.unique(true_labels_numeric)}")

Unique true labels (numeric): [0 1 2 3 4 5 6 7 8 9]


In [9]:
# Convert numeric labels to algorithm names using the dictionary
algorithm_names = {
    0: 'BIPOP-CMA-ES',
    1: 'CMA-CSA_Atamna',
    2: 'CMAES-APOP-KMA_Nguyen',
    3: 'DE-BFGS_voglis_noiseless',
    4: 'a-CMA-ES',
    5: 'ad-CMA-ES_Gissler',
    6: 'adm-CMA-ES_Gissler',
    7: 'dm-CMA-ES_Gissler',
    8: 's-CMA-ES_Gissler',
    9: 'sd-CMA-ES_Gissler'
}
true_labels = [algorithm_names.get(int(label), 'Unknown') for label in true_labels_numeric]

In [10]:
# Convert to numeric and handle missing values
for column in data_features.columns:
    data_features[column] = pd.to_numeric(data_features[column], errors='coerce')
data_features = data_features.fillna(0)

In [11]:
# Prepare features
X_new = data_features.values
print(f"Number of features: {X_new.shape[1]}")
X_new = X_new.astype(np.float32)

Number of features: 292


In [12]:
# Pad or trim features
n_features = X_new.shape[1]
if n_features < expected_features:
    padding = expected_features - n_features
    X_new = np.pad(X_new, ((0, 0), (0, padding)), mode='constant', constant_values=0)
elif n_features > expected_features:
    X_new = X_new[:, :expected_features]

In [13]:
# Reshape to match the model's input shape
X_new = X_new.reshape(-1, expected_side_length, expected_side_length, 1)

In [14]:
# Make predictions
predictions = model.predict(X_new, verbose=0)
predicted_classes = np.argmax(predictions, axis=1)

In [15]:
# Transform true labels using the label encoder
true_indices = label_encoder.transform(true_labels)

In [17]:
# Add predicted algorithms to output data
predicted_algorithms = [algorithm_names.get(pred, 'Unknown') for pred in predicted_classes]
output_data['Predicted Algorithm'] = predicted_algorithms

In [18]:
# Save to CSV with original values
output_data.to_csv('/content/drive/MyDrive/New-Job/predictions.csv', index=False)
print(f"Predictions saved to: /content/drive/MyDrive/New-Job/predictions.csv")

Predictions saved to: /content/drive/MyDrive/New-Job/predictions.csv
